07_merge_evictions_with_outcomes.ipynb

In [1]:
import dask_geopandas
import geopandas as gpd
import dask.dataframe as dd
import pandas as pd
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

INPUT_DATA_EVICTIONS_PARCELS_AND_TRACTS = "../../data/02_intermediate/evictions_with_parcels_and_tracts.csv"
INPUT_DATA_TAX_PARCELS_EAST = "../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_WEST.shp"
INPUT_DATA_TAX_PARCELS_WEST = "../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_EAST.shp"
INPUT_DATA_ZESTIMATES = "../../data/02_intermediate/zestimates.csv"
INPUT_DATA_CRIME = "../../data/01_raw/crime_incidents"
OUTPUT_DATA_UNRESTRICTED = "../../data/03_cleaned/unrestricted.csv"
OUTPUT_DATA_ZILLOW = "../../data/03_cleaned/zestimates_analysis.csv"
VERBOSE = True

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from Py

In [2]:
# Allocate resources.
cluster = SLURMCluster(queue='batch',
                       cores=4,
                       memory='8 GB')
cluster.scale(jobs=8)
client =  Client(cluster)

# Producing the Unrestricted Sample

### Merging evictions with crime incidents.

In [3]:
eviction_ddf = dd.read_csv(INPUT_DATA_EVICTIONS_PARCELS_AND_TRACTS).set_index('LOC_ID')
tax_parcel_columns_to_keep = ['LOC_ID', 'geometry']
eastern_dgdf = (dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS_EAST, columns=tax_parcel_columns_to_keep, npartitions=8)
                .dropna(subset='LOC_ID')
                .dissolve(by='LOC_ID'))
western_dgdf = (dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS_WEST, columns=tax_parcel_columns_to_keep, npartitions=8)
                .dropna(subset='LOC_ID')
                .dissolve(by='LOC_ID'))



ImportError: The 'read_file' function requires the 'pyogrio' package, but it is not installed or does not import correctly.
Importing pyogrio resulted in: No module named 'pyogrio'

In [7]:
western_dgdf

,MAP_PAR_ID,POLY_TYPE,SITE_ADDR,SHAPE_Leng,SHAPE_Area,geometry
npartitions=1,,,,,,
,object,object,object,float64,float64,geometry
,...,...,...,...,...,...


In [4]:
tax_parcels_dgdf = dd.multi.concat([eastern_dgdf, western_dgdf], axis=0)
dgdf = tax_parcels_dgdf.merge(eviction_ddf, how='right', left_index=True, right_index=True).compute()

KilledWorker: Attempted to run task ('aggregate-agg-3a1f09ac9ef549b5a32e412ff9fd7b40', 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://172.20.207.35:41874. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.